In [ ]:
# Use below line for demo in external colabs
#  !pip install -q git+https://github.com/nikitakapitan/transformers.git

In [2]:
from os.path import exists
from transformers.data.token import load_tokenizers
from transformers.data.vocab import load_vocab

from transformers.training.train import train

%load_ext autoreload
%autoreload 2

In [2]:
spacy_de, spacy_en = load_tokenizers()
vocab_src, vocab_tgt = load_vocab(spacy_de=spacy_de, spacy_en=spacy_en)

In [4]:
config = {
        'batch_size' : 32,
        'distributed' : False,
        'num_epochs' : 8,
        'accum_iter' : 10,
        'base_lr' : 1.0,
        'max_padding' : 72,
        'warmup' : 3000,
        'file_prefix' : 'multi30k_model_',
    }
model_path = 'multi30k_model_final.pt'

if not exists(model_path):
    train(
        vocab_src=vocab_src,
        vocab_tgt=vocab_tgt,
        spacy_de=spacy_de,
        spacy_en=spacy_en,
        config=config,
        )

Finished.
Vocabulary sizes:
len: SRC=8315 TGT=6384
